# Energy Load Data Cleaning Explanation

This notebook descrbes the process used to construct and clean the dataset.

Data was aquired from entsoe Transparency Platform at the following [link](https://transparency.entsoe.eu/load-domain/r2/totalLoadR2/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=09.08.2015+00:00|CET|DAY&biddingZone.values=CTY|10YES-REE------0!BZN|10YES-REE------0&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)#) (2015 data). Data is downloadable on an annual basis. this workbook constructs an example dataset using the years 2016-2018. The same functions may be used to construct any number of years available from this source.

Processes completed in the following functions:
1. format_data
    - renames the columns
    - shortens the text identifier for times
    - converts to a Datetime index
2. combine_annual_data
    - joins a dictionary if dataframes
3. interpolate_nans
    - fills the missing values using a linear interpolation method


In [126]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [62]:
path = './data/day-ahead-total-load-ES/'
files = ['Total Load - Day Ahead _ Actual_2016.csv', 'Total Load - Day Ahead _ Actual_2017.csv', 'Total Load - Day Ahead _ Actual_2018.csv']

In [63]:
#load in list of the datasets
data_sets = [pd.read_csv(path+file) for file in files]

In [125]:
#inital look at the unprocessed data
data_sets[0].head(24)

time  day_forecast  actual_load
0   01.01.2016 00:00       23273.0      22431.0
1   01.01.2016 01:00       22495.0      21632.0
2   01.01.2016 02:00       21272.0      20357.0
3   01.01.2016 03:00       20022.0      19152.0
4   01.01.2016 04:00       19148.0      18310.0
5   01.01.2016 05:00       18750.0      18054.0
6   01.01.2016 06:00       18729.0      18234.0
7   01.01.2016 07:00       18647.0      18596.0
8   01.01.2016 08:00       18242.0      18541.0
9   01.01.2016 09:00       18164.0      18942.0
10  01.01.2016 10:00       19478.0      20484.0
11  01.01.2016 11:00       20619.0      21805.0
12  01.01.2016 12:00       21366.0      22607.0
13  01.01.2016 13:00       22335.0      23178.0
14  01.01.2016 14:00       22670.0      23265.0
15  01.01.2016 15:00       21309.0      22061.0
16  01.01.2016 16:00       20767.0      21481.0
17  01.01.2016 17:00       21194.0      21830.0
18  01.01.2016 18:00       23845.0      24291.0
19  01.01.2016 19:00       25055.0      25234.0
20  01.01.2016 20:00       25922.0      25881.0
21  01.01.2016 21:00       26372.0      26149.0
22  01.01.2016 22:00       26374.0      25610.0
23  01.01.2016 23:00       24440.0      24000.0

### Formatting data

In [108]:
def format_data(data):
    '''
    Input: A dataframe of Day Ahead Total Load, and Actual Load obtained from csv data obtained from the entsoe Transparency Platform.
    
    Descrption:
    Input is a 3 column dataframe consisting of text time stamps with hourly frequency. 
    - Function formats the string in order to be formatted into a datetime.
    - Appends a datetime index and drops the time strings
    
    Output: A 2 column dataframe with a DatetimeIndex
    
    '''
    
    #set column names to something simple
    data.columns = ['time', 'day_forecast',
       'actual_load']

    #set the time to the first element in the time string. 
    #So 01.01.2018 00:00 - 01.01.2018 01:00 becomes 01.01.2018 00:00
    data['time'] = data['time'].str.split('-').apply(lambda x: x[0]).str.strip()
     
    #set the time strings to datetime obejects and set index as date time
    datetimes = pd.to_datetime(data['time'], format='%d-%m-%Y %H%M', errors='ignore')
    data_ = data.set_index(pd.DatetimeIndex(datetimes))
    
    #remove extra time column with original string objects
    data_time = data_[['day_forecast', 'actual_load']]
    
    return data_time

In [109]:
years = ['2016', '2017', '2018']

#create a dictionary of formatted pandas dataframes where key is each year
format_sets = {year: format_data(data_set) for year,data_set in zip(years, data_sets)}


In [111]:
#take a look at the formatted data
format_sets['2018'].head()

day_forecast  actual_load
time                                          
2018-01-01 00:00:00       23324.0      22779.0
2018-01-01 01:00:00       22688.0      22009.0
2018-01-01 02:00:00       21521.0      20589.0
2018-01-01 03:00:00       20294.0      19547.0
2018-01-01 04:00:00       19489.0      18871.0

### Combine data into single dataframe

In [112]:
def combine_annual_data(dictionary):
    """
    Input: a dictionary of dataframes.
    
    Output: a single dataframe
    """
    
    all_data_list = []
    
    for key in dictionary.keys():
        all_data_list.append(dictionary[key])
        
    data_all_years = pd.concat(all_data_list)
    
    return data_all_years

In [113]:
#combine into one single dataframe
data = combine_annual_data(clean_sets)

### Clean NANs

This data will be used for predicting day ahead energy demand. In dealing with nan values it is important not to change the structure of the data. 

Two ways this can occur:
   1. dropping values changes number of observations in a day. number of daily observations per day needs to line up with the days before and after.
   2. filling missing values with a single value (i.e. series mean value) is not representiative of the temporal nature of the data

In [71]:
#check for nans in the data
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26307 entries, 2016-01-01 00:00:00 to 2018-12-31 23:00:00
Data columns (total 2 columns):
day_forecast    26304 non-null float64
actual_load     26295 non-null float64
dtypes: float64(2)
memory usage: 616.6 KB


In [115]:
#count the total nans in each column and the total length of the data.
data.isnull().sum(), len(data)

(day_forecast     3
 actual_load     12
 dtype: int64, 26307)

A total of 3 and 12 nan values respectively in a dataset of length 26307. This is very clean data.

Can savfely imput the data with a linear interpolation function without changing the structure of the distribution.

In [116]:
#isolate the row indexes that with Nans
#use this to check the repalce_nans function works
nan_load = data[data['actual_load'].isnull()==True].index
nan_forecast = data[data['day_forecast'].isnull()==True].index
(nan_load, nan_forecast)

(DatetimeIndex(['2016-03-27 02:00:00', '2016-04-25 05:00:00',
                '2016-04-25 07:00:00', '2016-07-09 22:00:00',
                '2016-09-28 09:00:00', '2016-05-10 23:00:00',
                '2017-03-26 02:00:00', '2017-11-14 12:00:00',
                '2017-11-14 19:00:00', '2018-03-25 02:00:00',
                '2018-06-11 18:00:00', '2018-07-11 09:00:00'],
               dtype='datetime64[ns]', name='time', freq=None),
 DatetimeIndex(['2016-03-27 02:00:00', '2017-03-26 02:00:00',
                '2018-03-25 02:00:00'],
               dtype='datetime64[ns]', name='time', freq=None))

In [120]:
def interpolate_nans(data, columns):
    """
    Inputs:
    - data --- a dataframe of timeseries data
    - columns --- a list of column header names
    
    Process:
    Applies linear interpolation to fill the missing entries per column
    
    output: a dataframe
    """
    
    for col in columns:
        data[col] = data[col].interpolate(method='linear')
    
    return data

# data['day_forecast'] = data['day_forecast'].interpolate(method='linear')
# data['actual_load'] = data['actual_load'].interpolate(method='linear')

In [118]:
#check the function works
data[data['actual_load'].isnull()==True]

day_forecast  actual_load
time                                          
2016-03-27 02:00:00           NaN          NaN
2016-04-25 05:00:00       21471.0          NaN
2016-04-25 07:00:00       27635.0          NaN
2016-07-09 22:00:00       34985.0          NaN
2016-09-28 09:00:00       31072.0          NaN
2016-05-10 23:00:00       26641.0          NaN
2017-03-26 02:00:00           NaN          NaN
2017-11-14 12:00:00       33805.0          NaN
2017-11-14 19:00:00       35592.0          NaN
2018-03-25 02:00:00           NaN          NaN
2018-06-11 18:00:00       34752.0          NaN
2018-07-11 09:00:00       33938.0          NaN

In [123]:
#create list of the column headers
columns = data.columns[1:]

data = interpolate_nans(data, columns)

data[data['actual_load'].isnull()==True]

Empty DataFrame
Columns: [day_forecast, actual_load]
Index: []

In [124]:
#take a look at the interpreted values and see how they compare to the values around them.
for t in nan_load:
    print(data[str(t)])

                     day_forecast  actual_load
time                                          
2016-03-27 02:00:00       21388.0      21626.0
                     day_forecast  actual_load
time                                          
2016-04-25 05:00:00       21471.0      22528.0
                     day_forecast  actual_load
time                                          
2016-04-25 07:00:00       27635.0      26928.0
                     day_forecast  actual_load
time                                          
2016-07-09 22:00:00       34985.0      34263.5
                     day_forecast  actual_load
time                                          
2016-09-28 09:00:00       31072.0      31597.5
                     day_forecast  actual_load
time                                          
2016-05-10 23:00:00       26641.0      27519.5
                     day_forecast  actual_load
time                                          
2017-03-26 02:00:00       22504.0      22967.5
             

In [107]:
#export the prepared data as csv
data.to_csv(path + 'load_forecast_2016_2018.csv')